In [5]:
import pandas as pd
from transformers import pipeline
import spacy
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from collections import Counter
from thefuzz import fuzz
from warnings import simplefilter
import numpy as np
import math

In [3]:
# read and concat comment data into one dataframe
files = []
for i in range(7):
    filename = 'data/comments_game' + str(i + 1) + '.csv'
    df = pd.read_csv(filename, index_col=None, header=0)
    df['gameNum'] = i + 1
    files.append(df)

comments_df = pd.concat(files, axis=0, ignore_index=True)
comments_df

,body,category,communityName,createdAt,dataType,html,id,numberOfreplies,parentId,parsedId,postId,scrapedAt,upVotes,url,userId,username,gameNum
0,Hell of a finish,nba,r/nba,2025-07-06T02:30:37.000Z,comment,"&lt;div class=""md""&gt;&lt;p&gt;Hell of a finis...",t1_n1klkxl,0,t3_1l4dwhg,n1klkxl,t3_1l4dwhg,2025-08-24T04:28:52.834Z,1,https://www.reddit.com/r/nba/comments/1l4dwhg/...,t2_cyls0,balderdash9,1
1,I thought it was a ruff game. In other words i...,nba,r/nba,2025-06-07T05:18:56.000Z,comment,"&lt;div class=""md""&gt;&lt;p&gt;I thought it wa...",t1_mwg04ld,0,t3_1l4dwhg,mwg04ld,t3_1l4dwhg,2025-08-24T04:28:52.834Z,1,https://www.reddit.com/r/nba/comments/1l4dwhg/...,t2_cgihc4obw,Significant_Claim614,1
2,MVP??? Not! Choke!\nJoker was real MVP,nba,r/nba,2025-06-06T16:37:16.000Z,comment,"&lt;div class=""md""&gt;&lt;p&gt;MVP??? Not! Cho...",t1_mwcco3x,0,t3_1l4dwhg,mwcco3x,t3_1l4dwhg,2025-08-24T04:28:52.834Z,1,https://www.reddit.com/r/nba/comments/1l4dwhg/...,t2_1b22tzt359,Internal-Morning-499,1
3,"Wow just wow, go Pacers!!! Now if they adjust ...",nba,r/nba,2025-06-06T08:34:34.000Z,comment,"&lt;div class=""md""&gt;&lt;p&gt;Wow just wow, g...",t1_mwa7xgp,0,t3_1l4dwhg,mwa7xgp,t3_1l4dwhg,2025-08-24T04:28:52.834Z,2,https://www.reddit.com/r/nba/comments/1l4dwhg/...,t2_924h6hb0,Poopstiffler,1
4,Daigneault's coaching is still suspect. Why no...,nba,r/nba,2025-06-06T03:58:03.000Z,comment,"&lt;div class=""md""&gt;&lt;p&gt;Daigneault&amp;...",t1_mw9cml8,1,t3_1l4dwhg,mw9cml8,t3_1l4dwhg,2025-08-24T04:28:52.834Z,8,https://www.reddit.com/r/nba/comments/1l4dwhg/...,t2_1o07vg5n9k,Borbs_revenge_,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4773,How was that not an and 1? Smh,nba,r/nba,2025-06-23T02:30:15.000Z,comment,"&lt;div class=""md""&gt;&lt;p&gt;How was that no...",t1_mz9fwgl,0,t3_1li17ca,mz9fwgl,t3_1li17ca,2025-08-24T05:41:13.117Z,1,https://www.reddit.com/r/nba/comments/1li17ca/...,t2_w8u5wpoi4,Stainless711,7
4774,"What a pro wants, what a pro needs",nba,r/nba,2025-06-23T02:18:57.000Z,comment,"&lt;div class=""md""&gt;&lt;p&gt;What a pro want...",t1_mz9dz7s,0,t3_1li17ca,mz9dz7s,t3_1li17ca,2025-08-24T05:41:17.611Z,3,https://www.reddit.com/r/nba/comments/1li17ca/...,t2_3vva746q,rpgmgta,7
4775,He YouTube tv ads on YouTube tv drive me craY,nba,r/nba,2025-06-23T02:33:03.000Z,comment,"&lt;div class=""md""&gt;&lt;p&gt;He YouTube tv a...",t1_mz9gdkp,0,t3_1li17ca,mz9gdkp,t3_1li17ca,2025-08-24T05:41:17.615Z,1,https://www.reddit.com/r/nba/comments/1li17ca/...,t2_1646y1idze,Conscious_Log8495,7
4776,"r/nba rhetoric - ""OMG OKC are getting all the ...",nba,r/nba,2025-06-23T02:30:38.000Z,comment,"&lt;div class=""md""&gt;&lt;p&gt;&lt;a href=""/r/...",t1_mz9fyth,0,t3_1li17ca,mz9fyth,t3_1li17ca,2025-08-24T05:41:19.815Z,1,https://www.reddit.com/r/nba/comments/1li17ca/...,t2_34omb,inefekt,7


Scrape Basketball Reference for Statistics

In [16]:
#scrape overall stats
url = "https://www.basketball-reference.com/playoffs/2025-nba-finals-pacers-vs-thunder.html"
driver = webdriver.Chrome()
driver.implicitly_wait(10)
driver.get(url)

soup = BeautifulSoup(driver.page_source, 'html.parser')
table_types = ['OKC', 'OKCadvanced', 'IND', 'INDadvanced']
stat_df = pd.DataFrame()
adv_stat_df = pd.DataFrame()
for table_type in table_types:
    table = soup.find('table', id=table_type)
    temp_df = pd.read_html(str(table))[0]
    if (len(table_type) > 3):
        adv_stat_df = pd.concat([adv_stat_df, temp_df], ignore_index=True)
    else:
        stat_df = pd.concat([stat_df, temp_df], ignore_index=True)

stat_df = stat_df[stat_df['Unnamed: 1_level_0']['Player'] != 'Team Totals']
adv_stat_df = adv_stat_df[adv_stat_df['Unnamed: 1_level_0']['Player'] != 'Team Totals']
stat_df.to_csv('data/overall_stats.csv')
adv_stat_df.to_csv('data/overall_adv_stats.csv')

C:\Users\willi\AppData\Local\Temp\ipykernel_2332\1425733834.py:13: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  temp_df = pd.read_html(str(table))[0]
C:\Users\willi\AppData\Local\Temp\ipykernel_2332\1425733834.py:13: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  temp_df = pd.read_html(str(table))[0]
C:\Users\willi\AppData\Local\Temp\ipykernel_2332\1425733834.py:13: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  temp_df = pd.read_html(str(table))[0]
C:\Users\willi\AppData\Local\Temp\ipykernel_2332\1425733834.py:13: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version.

In [15]:
adv_stat_df

Unnamed: 0_level_0       Unnamed: 1_level_0 Unnamed: 2_level_0 Totals       \
                   Rk                   Player                Age      G   GS   
0                 1.0  Shai Gilgeous-Alexander               26.0      7  7.0   
1                 2.0           Jalen Williams               23.0      7  7.0   
2                 3.0            Chet Holmgren               22.0      7  7.0   
3                 4.0              Alex Caruso               30.0      7  0.0   
4                 5.0            Luguentz Dort               25.0      7  7.0   
5                 6.0            Cason Wallace               21.0      7  3.0   
6                 7.0            Aaron Wiggins               26.0      7  0.0   
7                 8.0       Isaiah Hartenstein               26.0      7  4.0   
8                 9.0               Isaiah Joe               25.0      5  0.0   
9                10.0          Jaylin Williams               22.0      3  0.0   
10               11.0         Kenrich Williams               30.0      7  0.0   
11               12.0             Dillon Jones               23.0      3  0.0   
12               13.0            Ajay Mitchell               22.0      4  0.0   
13               14.0            Ousmane Dieng               21.0      3  0.0   
14                NaN              Team Totals               24.9      7  NaN   
15                1.0            Pascal Siakam               30.0      7  7.0   
16                2.0        Tyrese Haliburton               24.0      7  7.0   
17                3.0       Bennedict Mathurin               22.0      7  0.0   
18                4.0           T.J. McConnell               32.0      7  0.0   
19                5.0          Andrew Nembhard               25.0      7  7.0   
20                6.0               Obi Toppin               26.0      7  0.0   
21                7.0             Myles Turner               28.0      7  7.0   
22                8.0            Aaron Nesmith               25.0      7  7.0   
23                9.0             Ben Sheppard               23.0      7  0.0   
24               10.0            Thomas Bryant               27.0      6  0.0   
25               11.0             Tony Bradley               27.0      3  0.0   
26               12.0            Johnny Furphy               20.0      4  0.0   
27               13.0            James Johnson               37.0      2  0.0   
28                NaN              Team Totals               26.1      7  NaN   

         Advanced                                                         \
      MP      TS%   eFG%  ORB%  DRB%  TRB%  AST%  STL%  BLK%  TOV%  USG%   
0    267    0.561  0.468   0.9  13.2   6.9  27.3   2.4   4.0  11.3  34.2   
1    243    0.541  0.465   3.3  13.5   8.3  19.2   1.2   0.0   9.0  29.6   
2    212    0.489  0.414   7.3  26.7  16.8   3.0   0.9   5.6  10.2  19.8   
3    202    0.583  0.528   4.9   9.1   7.0   7.8   4.2   3.1  11.6  14.6   
4    220    0.609  0.611   5.0  11.1   8.1   3.1   2.5   1.4   6.0   9.7   
5    156    0.540  0.539   3.5   9.6   6.6   4.5   3.8   2.6   9.3  11.8   
6    100    0.548  0.515   2.5  13.5   7.9   7.4   1.9   1.9  17.6  19.3   
7    134    0.649  0.609  15.4  24.8  19.8  17.6   2.2   1.4  18.9  11.9   
8     41    0.838  0.808   0.0   8.5   4.4   5.5   1.2   0.0  12.3  17.3   
9     15    1.025  1.125   0.0  42.2  22.9  14.1   3.2   0.0  17.0  17.8   
10    55    0.322  0.350  11.0  14.2  12.9   9.6   0.0   2.1  15.5  10.2   
11    10    1.000  1.000  12.5  10.2  11.2  27.4   0.0   0.0  33.3  13.8   
12    19    0.254  0.143  14.0  11.5  13.8  42.8   0.0   0.0  27.6  24.6   
13     6    0.500  0.500   0.0   0.0   0.0   0.0   0.0  17.1   0.0  24.2   
14  1680    0.557  0.497  25.7  74.1  49.3  47.5  10.9  12.4  12.3   NaN   
15   234    0.574  0.505   7.7  18.6  13.1  19.4   2.3   3.5   9.3  23.8   
16   210    0.580  0.556   1.1  15.4   8.5  31.5   1.8   1.7  20.7  21.8   
17   144    0.580  0.482  14.4  14.3  13.9  11.

In [ ]:
#scrape individual game stats
urls = ['https://www.basketball-reference.com/boxscores/202506050OKC.html',
        'https://www.basketball-reference.com/boxscores/202506080OKC.html',
        'https://www.basketball-reference.com/boxscores/202506110IND.html',
        'https://www.basketball-reference.com/boxscores/202506130IND.html',
        'https://www.basketball-reference.com/boxscores/202506160OKC.html',
        'https://www.basketball-reference.com/boxscores/202506190IND.html',
        'https://www.basketball-reference.com/boxscores/202506220OKC.html']

driver = webdriver.Chrome()
driver.command_executor.set_timeout(1000)
game_stat_df = pd.DataFrame()
game_adv_stat_df = pd.DataFrame()
for i in range(len(urls)):
    driver.implicitly_wait(10)
    driver.get(urls[i])

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    table_types = ['box-OKC-game-basic', 'box-OKC-game-advanced', 'box-IND-game-basic', 'box-IND-game-advanced']

    for table_type in table_types:
        table = soup.find('table', id=table_type)
        temp_df = pd.read_html(str(table))[0]
        temp_df['gameNum'] = i + 1
        if (len(table_type) > 18):
            game_adv_stat_df = pd.concat([game_adv_stat_df, temp_df], ignore_index=True)
        else:
            game_stat_df = pd.concat([game_stat_df, temp_df], ignore_index=True)

game_stat_df
game_adv_stat_df

C:\Users\willi\AppData\Local\Temp\ipykernel_29052\3891678162.py:11: DeprecationWarning: set_timeout() in RemoteConnection is deprecated, set timeout in client_config instead
  driver.command_executor.set_timeout(1000)
C:\Users\willi\AppData\Local\Temp\ipykernel_29052\3891678162.py:23: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  temp_df = pd.read_html(str(table))[0]
C:\Users\willi\AppData\Local\Temp\ipykernel_29052\3891678162.py:23: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  temp_df = pd.read_html(str(table))[0]
C:\Users\willi\AppData\Local\Temp\ipykernel_29052\3891678162.py:23: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO'

Unnamed: 0_level_0 Advanced Box Score Stats                 \
                    Starters                       MP            TS%   
0    Shai Gilgeous-Alexander                    39:34           .567   
1             Jalen Williams                    36:27           .409   
2              Luguentz Dort                    36:21           .750   
3              Cason Wallace                    33:16           .333   
4              Chet Holmgren                    23:31           .304   
..                       ...                      ...            ...   
226            Johnny Furphy                     0:32            NaN   
227           Isaiah Jackson            Did Not Dress  Did Not Dress   
228            James Johnson             Did Not Play   Did Not Play   
229            Jarace Walker            Did Not Dress  Did Not Dress   
230              Team Totals                      240           .550   

                                                                 \
              eFG%           3PAr            FTr           ORB%   
0             .517           .200           .267            0.0   
1             .342           .211           .211            2.5   
2             .750           .900           .000            5.0   
3             .333           .333           .000            0.0   
4             .222           .111           .222            0.0   
..             ...            ...            ...            ...   
226            NaN            NaN            NaN            0.0   
227  Did Not Dress  Did Not Dress  Did Not Dress  Did Not Dress   
228   Did Not Play   Did Not Play   Did Not Play   Did Not Play   
229  Did Not Dress  Did Not Dress  Did Not Dress  Did Not Dress   
230           .493           .400           .414           30.8   

                                                                 \
              DRB%           TRB%           AST%           STL%   
0             14.4            6.4           16.5            3.6   
1              9.4            5.5           25.4            1.3   
2              6.3            5.6            4.1            5.2   
3              6.9            3.0            4.2            1.4   
4             29.2           12.9            0.0            0.0   
..             ...            ...            ...            ...   
226            0.0            0.0            0.0            0.0   
227  Did Not Dress  Did Not Dress  Did Not Dress  Did Not Dress   
228   Did Not Play   Did Not Play   Did Not Play   Did Not Play   
229  Did Not Dress  Did Not Dress  Did Not Dress  Did Not Dress   
230           71.7           52.9           58.6            6.6   

                                                                 \
              BLK%           TOV%           USG%           ORtg   
0              0.0            5.6           37.3            114   
1              0.0            0.0           23.7            102   
2              6.1            0.0           11.4            158   
3              3.4            0.0           11.2             76   
4              4.7            9.2           19.2             60   
..             ...            ...            ...            ...   
226            0.0            NaN            0.0              0   
227  Did Not Dress  Did Not Dress  Did Not Dress  Did Not Dress   
228   Did Not Play   Did Not Play   Did Not Play   Did Not Play   
229  Did Not Dress  Did Not Dress  Did Not Dress  Did Not Dress   
230            8.5           21.7          100.0           99.6   

                                  gameNum  
              DRtg            BPM          
0              108           10.4       1  
1              115           -1.2       1  
2              104           12.3       1  
3              114           -6.0       1  
4              110          -10.9       1  
..             ...            ...     ...  
226            123           -2.6       7  
227  Did Not Dress  Did Not Dress       7  
228   Did Not Pla

Sentiment Analysis & POS Tagging

In [4]:
!python -m spacy download en_core_web_sm

     --------------------------------------- 12.8/12.8 MB 28.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')



[notice] A new release of pip available: 22.3 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
# load models
nlp = spacy.load('en_core_web_sm')
sentiment_analysis = pipeline("sentiment-analysis", model="siebert/sentiment-roberta-large-english", max_length=512, truncation=True)

Device set to use cpu


In [8]:
nickname_dict = {}
for name in stat_df['Unnamed: 1_level_0']['Player']:
    if not (name == 'Team Totals'):
        names = name.split(" ")
        nicknames = []
        if name == 'Tyrese Haliburton':
            nicknames = ['Hali', 'Haliban']
        elif name == 'Jalen Williams':
            nicknames = ['JDub']
        elif name == 'Shai Gilgeous-Alexander':
            nicknames = ['SGA']
        elif name == 'Pascal Siakam':
            nicknames = ['Spicy P']
        elif name == 'Jaylin Williams':
            nicknames = ['JWill']
        elif name == 'Luguentz Dort':
            nicknames = ['Lu', 'Dorture Chamber']
        nickname_dict[name] = names + nicknames 

In [9]:
for index, row in comments_df.iterrows():
    # match player associated with comment
    doc = nlp(row['body'])
    names = set()
    for token in doc:
        if token.pos_ == 'PROPN':
            globalMax = -1
            name = ""
            for key, value in nickname_dict.items():
                nameMax = -1
                for i in range(len(value)):
                    similarity_ratio = fuzz.ratio(value[i], token.text)
                    nameMax = max(nameMax, similarity_ratio)
                name = key if nameMax > globalMax else name
                globalMax = max(globalMax, nameMax)
            if globalMax > 80:
                names.add(name)
    
    comments_df.loc[index, 'aboutPlayer'] = True if len(names) > 0 else False
    comments_df.loc[index, 'players'] = str(names)[1:-1] if len(names) > 0 else ''

comments_df


,body,category,communityName,createdAt,dataType,html,id,numberOfreplies,parentId,parsedId,postId,scrapedAt,upVotes,url,userId,username,gameNum,aboutPlayer,players
0,Hell of a finish,nba,r/nba,2025-07-06T02:30:37.000Z,comment,"&lt;div class=""md""&gt;&lt;p&gt;Hell of a finis...",t1_n1klkxl,0,t3_1l4dwhg,n1klkxl,t3_1l4dwhg,2025-08-24T04:28:52.834Z,1,https://www.reddit.com/r/nba/comments/1l4dwhg/...,t2_cyls0,balderdash9,1,False,
1,I thought it was a ruff game. In other words i...,nba,r/nba,2025-06-07T05:18:56.000Z,comment,"&lt;div class=""md""&gt;&lt;p&gt;I thought it wa...",t1_mwg04ld,0,t3_1l4dwhg,mwg04ld,t3_1l4dwhg,2025-08-24T04:28:52.834Z,1,https://www.reddit.com/r/nba/comments/1l4dwhg/...,t2_cgihc4obw,Significant_Claim614,1,False,
2,MVP??? Not! Choke!\nJoker was real MVP,nba,r/nba,2025-06-06T16:37:16.000Z,comment,"&lt;div class=""md""&gt;&lt;p&gt;MVP??? Not! Cho...",t1_mwcco3x,0,t3_1l4dwhg,mwcco3x,t3_1l4dwhg,2025-08-24T04:28:52.834Z,1,https://www.reddit.com/r/nba/comments/1l4dwhg/...,t2_1b22tzt359,Internal-Morning-499,1,False,
3,"Wow just wow, go Pacers!!! Now if they adjust ...",nba,r/nba,2025-06-06T08:34:34.000Z,comment,"&lt;div class=""md""&gt;&lt;p&gt;Wow just wow, g...",t1_mwa7xgp,0,t3_1l4dwhg,mwa7xgp,t3_1l4dwhg,2025-08-24T04:28:52.834Z,2,https://www.reddit.com/r/nba/comments/1l4dwhg/...,t2_924h6hb0,Poopstiffler,1,False,
4,Daigneault's coaching is still suspect. Why no...,nba,r/nba,2025-06-06T03:58:03.000Z,comment,"&lt;div class=""md""&gt;&lt;p&gt;Daigneault&amp;...",t1_mw9cml8,1,t3_1l4dwhg,mw9cml8,t3_1l4dwhg,2025-08-24T04:28:52.834Z,8,https://www.reddit.com/r/nba/comments/1l4dwhg/...,t2_1o07vg5n9k,Borbs_revenge_,1,True,'Cason Wallace'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4773,How was that not an and 1? Smh,nba,r/nba,2025-06-23T02:30:15.000Z,comment,"&lt;div class=""md""&gt;&lt;p&gt;How was that no...",t1_mz9fwgl,0,t3_1li17ca,mz9fwgl,t3_1li17ca,2025-08-24T05:41:13.117Z,1,https://www.reddit.com/r/nba/comments/1li17ca/...,t2_w8u5wpoi4,Stainless711,7,False,
4774,"What a pro wants, what a pro needs",nba,r/nba,2025-06-23T02:18:57.000Z,comment,"&lt;div class=""md""&gt;&lt;p&gt;What a pro want...",t1_mz9dz7s,0,t3_1li17ca,mz9dz7s,t3_1li17ca,2025-08-24T05:41:17.611Z,3,https://www.reddit.com/r/nba/comments/1li17ca/...,t2_3vva746q,rpgmgta,7,False,
4775,He YouTube tv ads on YouTube tv drive me craY,nba,r/nba,2025-06-23T02:33:03.000Z,comment,"&lt;div class=""md""&gt;&lt;p&gt;He YouTube tv a...",t1_mz9gdkp,0,t3_1li17ca,mz9gdkp,t3_1li17ca,2025-08-24T05:41:17.615Z,1,https://www.reddit.com/r/nba/comments/1li17ca/...,t2_1646y1idze,Conscious_Log8495,7,False,
4776,"r/nba rhetoric - ""OMG OKC are getting all the ...",nba,r/nba,2025-06-23T02:30:38.000Z,comment,"&lt;div class=""md""&gt;&lt;p&gt;&lt;a href=""/r/...",t1_mz9fyth,0,t3_1li17ca,mz9fyth,t3_1li17ca,2025-08-24T05:41:19.815Z,1,https://www.reddit.com/r/nba/comments/1li17ca/...,t2_34omb,inefekt,7,False,


In [10]:
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

for index, row in comments_df.iterrows():
    analysis = sentiment_analysis(row['body'])[0]
    comments_df.loc[index, 'sentiment'] = analysis['label']
    comments_df.loc[index, 'score'] = analysis['score']

comments_df

,body,category,communityName,createdAt,dataType,html,id,numberOfreplies,parentId,parsedId,...,scrapedAt,upVotes,url,userId,username,gameNum,aboutPlayer,players,sentiment,score
0,Hell of a finish,nba,r/nba,2025-07-06T02:30:37.000Z,comment,"&lt;div class=""md""&gt;&lt;p&gt;Hell of a finis...",t1_n1klkxl,0,t3_1l4dwhg,n1klkxl,...,2025-08-24T04:28:52.834Z,1,https://www.reddit.com/r/nba/comments/1l4dwhg/...,t2_cyls0,balderdash9,1,False,,POSITIVE,0.998576
1,I thought it was a ruff game. In other words i...,nba,r/nba,2025-06-07T05:18:56.000Z,comment,"&lt;div class=""md""&gt;&lt;p&gt;I thought it wa...",t1_mwg04ld,0,t3_1l4dwhg,mwg04ld,...,2025-08-24T04:28:52.834Z,1,https://www.reddit.com/r/nba/comments/1l4dwhg/...,t2_cgihc4obw,Significant_Claim614,1,False,,POSITIVE,0.997967
2,MVP??? Not! Choke!\nJoker was real MVP,nba,r/nba,2025-06-06T16:37:16.000Z,comment,"&lt;div class=""md""&gt;&lt;p&gt;MVP??? Not! Cho...",t1_mwcco3x,0,t3_1l4dwhg,mwcco3x,...,2025-08-24T04:28:52.834Z,1,https://www.reddit.com/r/nba/comments/1l4dwhg/...,t2_1b22tzt359,Internal-Morning-499,1,False,,NEGATIVE,0.991552
3,"Wow just wow, go Pacers!!! Now if they adjust ...",nba,r/nba,2025-06-06T08:34:34.000Z,comment,"&lt;div class=""md""&gt;&lt;p&gt;Wow just wow, g...",t1_mwa7xgp,0,t3_1l4dwhg,mwa7xgp,...,2025-08-24T04:28:52.834Z,2,https://www.reddit.com/r/nba/comments/1l4dwhg/...,t2_924h6hb0,Poopstiffler,1,False,,POSITIVE,0.998161
4,Daigneault's coaching is still suspect. Why no...,nba,r/nba,2025-06-06T03:58:03.000Z,comment,"&lt;div class=""md""&gt;&lt;p&gt;Daigneault&amp;...",t1_mw9cml8,1,t3_1l4dwhg,mw9cml8,...,2025-08-24T04:28:52.834Z,8,https://www.reddit.com/r/nba/comments/1l4dwhg/...,t2_1o07vg5n9k,Borbs_revenge_,1,True,'Cason Wallace',NEGATIVE,0.999490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4773,How was that not an and 1? Smh,nba,r/nba,2025-06-23T02:30:15.000Z,comment,"&lt;div class=""md""&gt;&lt;p&gt;How was that no...",t1_mz9fwgl,0,t3_1li17ca,mz9fwgl,...,2025-08-24T05:41:13.117Z,1,https://www.reddit.com/r/nba/comments/1li17ca/...,t2_w8u5wpoi4,Stainless711,7,False,,NEGATIVE,0.997393
4774,"What a pro wants, what a pro needs",nba,r/nba,2025-06-23T02:18:57.000Z,comment,"&lt;div class=""md""&gt;&lt;p&gt;What a pro want...",t1_mz9dz7s,0,t3_1li17ca,mz9dz7s,...,2025-08-24T05:41:17.611Z,3,https://www.reddit.com/r/nba/comments/1li17ca/...,t2_3vva746q,rpgmgta,7,False,,POSITIVE,0.997588
4775,He YouTube tv ads on YouTube tv drive me craY,nba,r/nba,2025-06-23T02:33:03.000Z,comment,"&lt;div class=""md""&gt;&lt;p&gt;He YouTube tv a...",t1_mz9gdkp,0,t3_1li17ca,mz9gdkp,...,2025-08-24T05:41:17.615Z,1,https://www.reddit.com/r/nba/comments/1li17ca/...,t2_1646y1idze,Conscious_Log8495,7,False,,NEGATIVE,0.996749
4776,"r/nba rhetoric - ""OMG OKC are getting all the ...",nba,r/nba,2025-06-23T02:30:38.000Z,comment,"&lt;div class=""md""&gt;&lt;p&gt;&lt;a href=""/r/...",t1_mz9fyth,0,t3_1li17ca,mz9fyth,...,2025-08-24T05:41:19.815Z,1,https://www.reddit.com/r/nba/comments/1li17ca/...,t2_34omb,inefekt,7,False,,NEGATIVE,0.999147


In [11]:
comments_df.to_csv('data/comments_final.csv')

In [6]:
adv_stats = pd.read_csv('data/overall_adv_stats.csv')
league_avg = 114.5
for index, row in adv_stats.iterrows():
    if row['Unnamed: 1_level_0'] != 'Player':
        net_rating = (float(row['Advanced.10']) - league_avg) + (league_avg - float(row['Advanced.11']))
        adv_stats.loc[index, 'NetRtg'] = net_rating

sample_var = np.var(adv_stats['NetRtg'].values[1:]) 
sample_sd = math.sqrt(sample_var)

for index, row in adv_stats.iterrows():
    if row['Unnamed: 1_level_0'] != 'Player':
        studentized_score = row['NetRtg'] / sample_sd # population mean of net rating is, by definition, 0
        adv_stats.loc[index, 'StudentizedScr'] = studentized_score

adv_stats

,Unnamed: 0,Unnamed: 0_level_0,Unnamed: 1_level_0,Unnamed: 2_level_0,Totals,Totals.1,Totals.2,Advanced,Advanced.1,Advanced.2,...,Advanced.5,Advanced.6,Advanced.7,Advanced.8,Advanced.9,Advanced.10,Advanced.11,Advanced.12,NetRtg,StudentizedScr
0,NaN,Rk,Player,Age,G,GS,MP,TS%,eFG%,ORB%,...,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,GmSc,NaN,NaN
1,0.0,1.0,Shai Gilgeous-Alexander,26.0,7,7.0,267,0.561,0.468,0.9,...,27.3,2.4,4.0,11.3,34.2,114.0,108.0,21.8,6.0,0.260388
2,1.0,2.0,Jalen Williams,23.0,7,7.0,243,0.541,0.465,3.3,...,19.2,1.2,0.0,9.0,29.6,112.0,113.0,15.3,-1.0,-0.043398
3,2.0,3.0,Chet Holmgren,22.0,7,7.0,212,0.489,0.414,7.3,...,3.0,0.9,5.6,10.2,19.8,100.0,108.0,9.4,-8.0,-0.347184
4,3.0,4.0,Alex Caruso,30.0,7,0.0,202,0.583,0.528,4.9,...,7.8,4.2,3.1,11.6,14.6,117.0,106.0,9.4,11.0,0.477378
5,4.0,5.0,Luguentz Dort,25.0,7,7.0,220,0.609,0.611,5.0,...,3.1,2.5,1.4,6.0,9.7,127.0,110.0,7.1,17.0,0.737767
6,5.0,6.0,Cason Wallace,21.0,7,3.0,156,0.54,0.539,3.5,...,4.5,3.8,2.6,9.3,11.8,102.0,107.0,5.2,-5.0,-0.216990
7,6.0,7.0,Aaron Wiggins,26.0,7,0.0,100,0.548,0.515,2.5,...,7.4,1.9,1.9,17.6,19.3,94.0,110.0,3.5,-16.0,-0.694369
8,7.0,8.0,Isaiah Hartenstein,26.0,7,4.0,134,0.649,0.609,15.4,...,17.6,2.2,1.4,18.9,11.9,134.0,107.0,7.9,27.0,1.171747
9,8.0,9.0,Isaiah Joe,25.0,5,0.0,41,0.838,0.808,0.0,...,5.5,1.2,0.0,12.3,17.3,134.0,114.0,3.3,20.0,0.867961


In [7]:
comments = pd.read_csv('data/comments_final.csv')
comment_count_df = pd.DataFrame()
comment_count_df['Player'] = adv_stats['Unnamed: 1_level_0'][1:]
comment_count_df['PosCount'] = 0
comment_count_df['NegCount'] = 0

for index, row, in comments.iterrows():
    if row['aboutPlayer'] == True:
        for player in str(row['players'])[1:-1].split(', '):
            index = comment_count_df.index[comment_count_df['Player'] == player]
            if row['sentiment'] == 'POSITIVE':
                comment_count_df.loc[index, 'PosCount'] = comment_count_df['PosCount'] + 1
            else:
                comment_count_df.loc[index, 'NegCount'] = comment_count_df['NegCount'] + 1
comment_count_df['Ratio'] = comment_count_df['PosCount'] / comment_count_df['NegCount']
sample_var = np.var(comment_count_df['Ratio'].dropna().values) 
sample_sd = math.sqrt(sample_var)
mean_ratio = comment_count_df['Ratio'].sum() / comment_count_df.shape[0]

for index, row in comment_count_df.iterrows():
    studentized_score = (row['Ratio'] - mean_ratio) / sample_sd 
    comment_count_df.loc[index, 'CommentStudentizedScore'] = studentized_score

comment_count_df


,Player,PosCount,NegCount,Ratio,CommentStudentizedScore
1,Shai Gilgeous-Alexander,50,96,0.520833,-0.033112
2,Jalen Williams,26,29,0.896552,1.025922
3,Chet Holmgren,4,19,0.210526,-0.907771
4,Alex Caruso,11,21,0.523810,-0.024723
5,Luguentz Dort,5,21,0.238095,-0.830063
6,Cason Wallace,1,1,1.000000,1.317511
7,Aaron Wiggins,5,5,1.000000,1.317511
8,Isaiah Hartenstein,0,1,0.000000,-1.501179
9,Isaiah Joe,0,1,0.000000,-1.501179
10,Jaylin Williams,1,2,0.500000,-0.091834


In [8]:
comment_count_df['StatStudentizedScr'] = adv_stats['StudentizedScr']
comment_count_df['Difference'] = comment_count_df['CommentStudentizedScore'] - comment_count_df['StatStudentizedScr']
comment_count_df

,Player,PosCount,NegCount,Ratio,CommentStudentizedScore,StatStudentizedScr,Difference
1,Shai Gilgeous-Alexander,50,96,0.520833,-0.033112,0.260388,-0.293500
2,Jalen Williams,26,29,0.896552,1.025922,-0.043398,1.069320
3,Chet Holmgren,4,19,0.210526,-0.907771,-0.347184,-0.560587
4,Alex Caruso,11,21,0.523810,-0.024723,0.477378,-0.502101
5,Luguentz Dort,5,21,0.238095,-0.830063,0.737767,-1.567829
6,Cason Wallace,1,1,1.000000,1.317511,-0.216990,1.534501
7,Aaron Wiggins,5,5,1.000000,1.317511,-0.694369,2.011879
8,Isaiah Hartenstein,0,1,0.000000,-1.501179,1.171747,-2.672926
9,Isaiah Joe,0,1,0.000000,-1.501179,0.867961,-2.369140
10,Jaylin Williams,1,2,0.500000,-0.091834,2.083106,-2.174940


In [10]:
comment_count_df.to_csv('data/comment_stat_comparison.csv')